In [1]:
import torch
import numpy as np
import random
import os
random_seed = 1
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
seed_torch(random_seed)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('/home/xiaoguzai/模型/Longformer')

import pandas as pd
df = pd.read_csv('/home/xiaoguzai/数据/Evaluate student writing/train.csv')
#df = pd.read_csv('/home/xiaoguzai/数据/Try Evaluate student writing/train.csv')
print(df.shape)

IDS = df.id.unique()
print('There are',len(IDS),'train texts.')

MAX_LEN = 2048

# THE TOKENS AND ATTENTION ARRAYS
train_tokens = np.zeros((len(IDS),MAX_LEN), dtype='int32')
train_attention = np.zeros((len(IDS),MAX_LEN), dtype='int32')

# THE 14 CLASSES FOR NER
lead_b = np.zeros((len(IDS),MAX_LEN))
lead_i = np.zeros((len(IDS),MAX_LEN))

position_b = np.zeros((len(IDS),MAX_LEN))
position_i = np.zeros((len(IDS),MAX_LEN))

evidence_b = np.zeros((len(IDS),MAX_LEN))
evidence_i = np.zeros((len(IDS),MAX_LEN))

claim_b = np.zeros((len(IDS),MAX_LEN))
claim_i = np.zeros((len(IDS),MAX_LEN))

conclusion_b = np.zeros((len(IDS),MAX_LEN))
conclusion_i = np.zeros((len(IDS),MAX_LEN))

counterclaim_b = np.zeros((len(IDS),MAX_LEN))
counterclaim_i = np.zeros((len(IDS),MAX_LEN))

rebuttal_b = np.zeros((len(IDS),MAX_LEN))
rebuttal_i = np.zeros((len(IDS),MAX_LEN))

# HELPER VARIABLES
train_lens = []
targets_b = [lead_b, position_b, evidence_b, claim_b, conclusion_b, counterclaim_b, rebuttal_b]
#这里面放入targets_b的时候已经想到了文章的数目:np.zeros((len(IDS),MAX_LEN)),len(IDS):文章数目
targets_i = [lead_i, position_i, evidence_i, claim_i, conclusion_i, counterclaim_i, rebuttal_i]
target_map = {'Lead':0, 'Position':1, 'Evidence':2, 'Claim':3, 'Concluding Statement':4,
             'Counterclaim':5, 'Rebuttal':6}
train_text = []
train_offset = []

from tqdm import tqdm
for id_num in tqdm(range(len(IDS))):
    #if id_num%100==0: print(id_num,', ',end='')
        
    # READ TRAIN TEXT, TOKENIZE, AND SAVE IN TOKEN ARRAYS    
    n = IDS[id_num]
    name = f'/home/xiaoguzai/数据/Evaluate student writing/train/{n}.txt'
    txt = open(name, 'r').read()
    train_lens.append(len(txt.split()))
    tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
    train_tokens[id_num,] = tokens['input_ids']
    train_attention[id_num,] = tokens['attention_mask']
    # FIND TARGETS IN TEXT AND SAVE IN TARGET ARRAYS
    offsets = tokens['offset_mapping']
    #读取文件名称并切分，获取tokens['input_ids'],
    #tokens['attention_mask']以及tokens['offset_mapping']内容
    train_offset.append(offsets)
    train_text.append(txt)
    offset_index = 0
    df_current = df.loc[df.id==n]
    for index,row in df_current.iterrows():
    #!!!这里有bug!!!应该为df_current.iterrows，而不是df.iterrows
        a = row.discourse_start
        b = row.discourse_end
        #discourse_start为开始的字母，从第8个字母开始，discourse_end为结束的字母，从第229个字母开始
        #这个if要放在循环外面，节约运行时间
        if offset_index > len(offsets)-1:
            break
        c = offsets[offset_index][0]
        d = offsets[offset_index][1]
        #切分出来带空格的第几个字母
        beginning = True
        while b>c:
            if (a<=c)&(d<=b):
                k = target_map[row.discourse_type]            
                r"""
                这里a,b,c,d实际上的大小关系
                (a           ,         b)
                (c<---------c,d------->d)
                (c,d)包含在(a,b)之中，证明这个字母所在的单词为这个标签的内容
                c,d为字母编号，id_num为划分的id编号(单词编号)
                """
                #“Lead","Position","Evidence","Claim","Concluding Statement",
                if beginning:
                    targets_b[k][id_num][offset_index] = 1
                    beginning = False
                else:
                    targets_i[k][id_num][offset_index] = 1
                #如果对应字母在这个(a,d)的范围之中，标记当前分类类别
                #这里不需要offset_index+1,因为起始为(0,0)代表打头的标记了
                #标记当前的单词即可
            offset_index += 1
            if offset_index>len(offsets)-1:
                break
            c = offsets[offset_index][0]
            d = offsets[offset_index][1]
            #k是分类,id_num为对应的id,offset_index为对应的predictstring

targets = np.zeros((len(IDS),MAX_LEN))
for k in range(7):
    targets[:,:] += targets_b[k]*(2*k+1)
    targets[:,:] += targets_i[k]*(2*k+2)
#1~14
targets[:,:] += (targets[:,:] == 0)*15
#other:15,1~15
targets[:,:] = targets[:,:]-1
#targets[:,:] -= (train_tokens[:,:] == 1)*(targets[:,:]+100)
#0~14,padding=14
#torch crossentropy无用label只能用-100替代

from pythonicforbert import get_model_function
import json
with open('/home/xiaoguzai/模型/Longformer/config.json','r',encoding='utf8') as fp:
    json_data = json.load(fp)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#config:1e-7
import torch.nn as nn
class ClassificationModel(nn.Module):
    def __init__(self,model,config,n_labels):
        super(ClassificationModel,self).__init__()
        self.model = model
        self.fc = nn.Linear(config.embedding_size,15)
        
    def forward(self,input_ids,attention_mask):
        #mask_ids = torch.not_equal(input_ids,1)
        #英文roberta padding=1
        output = self.model(input_ids,attention_mask=attention_mask)
        #output = nn.Dropout(0.1)(output)
        output = self.fc(output)
        
        #output = torch.softmax(output,dim=-1)
        
        return output

from torch.utils.data import Dataset,DataLoader
class TrainDataset(Dataset):
    def __init__(self,ids,tokens,text,offset_mapping,attention_mask,targets):
        #id,tokens:input_id,text,offset_mapping,
        self.ids = ids
        self.tokens = tokens
        self.text = text
        self.attention_mask = attention_mask
        self.offset_mapping = offset_mapping
        self.targets = targets
        #前面的tokens = tokenizer.encode_plus(txt, max_length=MAX_LEN, padding='max_length',
        #truncation=True, return_offsets_mapping=True)会将超出max_length的部分去除掉
        self.tensors = [self.ids,\
                        torch.tensor(self.tokens,dtype=torch.long),\
                        self.text,\
                        self.offset_mapping,\
                        torch.tensor(self.attention_mask,dtype=torch.long),\
                        torch.tensor(self.targets,dtype=torch.long)]
    
    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self,index):
        return tuple(tensor[index] for tensor in self.tensors)

proba_thresh = {
    "Lead": 0.7,
    "Position": 0.55,
    "Evidence": 0.65,
    "Claim": 0.55,
    "Concluding Statement": 0.7,
    "Counterclaim": 0.5,
    "Rebuttal": 0.55,
}
#定义了每一种标记的门槛

min_thresh = {
    "Lead": 9,
    "Position": 5,
    "Evidence": 14,
    "Claim": 3,
    "Concluding Statement": 11,
    "Counterclaim": 6,
    "Rebuttal": 4,
}

def change_id_to_predictstring(test_samples):
    temp_df = []
    for sample_idx, sample in enumerate(test_samples):
        preds = sample["preds"]
        offset_mapping = sample["offset_mapping"]
        sample_id = sample["id"]
        sample_text = sample["text"]
        sample_input_ids = sample["input_ids"]
        sample_pred_scores = sample["pred_scores"]
        sample_preds = []
        if len(preds) < len(offset_mapping):
            #没有出现过这种情况
            preds = preds + ["O"] * (len(offset_mapping) - len(preds))
            sample_pred_scores = sample_pred_scores + [0] * (len(offset_mapping) - len(sample_pred_scores))
            #这里没看到下面有用到O来填充的
        idx = 0
        phrase_preds = []
        while idx < len(offset_mapping):
            start, end = offset_mapping[idx]
            if preds[idx] != "O":
                label = preds[idx][2:]
                #不为O的情况下，比如标签为B-Lead
                #这里取出从2开始的数值Lead
            else:
                label = "O"
            phrase_scores = []
            phrase_scores.append(sample_pred_scores[idx])
            #这里phrase_scores压入的是刚开始的得分
            #phrase_scores保存每一层概率的得分
            idx += 1
            while idx < len(offset_mapping):
                if label == "O":
                    matching_label = "O"
                else:
                    matching_label = f"I-{label}"
                if preds[idx] == matching_label:
                    _, end = offset_mapping[idx]
                    phrase_scores.append(sample_pred_scores[idx])
                    idx += 1
                else:
                    break
            #取出后面的label与当前label相同的标签内容
            #比如前面的为Lead，这里依次找寻I-Lead(I-Lead均为匹配直到不匹配为止)
            if "end" in locals():
                phrase = sample_text[start:end]
                phrase_preds.append((phrase, start, end, label, phrase_scores))
            #print('$$$end = $$$')
            #print(end)
            r"""
            phrase为当前的字符串，phrase_preds压入的为现在得到的得分
            ***phrase_preds = ***
            [("80% of Americans believe seeking multiple opinions 
            can help them make better choices, and for good reason. 
            .............
            In all of these stories, the hero listens to other people, 
            and benefited from the people around them being more knowledgeable, 
            more experienced, and giving the hero more choices to consider.", 
            0, 752, 'Lead', [0.9794921875, 0.98046875, 0.98193359375, 0.98291015625, 
            0.97900390625, 0.9794921875, 0.9814453125, 0.9814453125, 0.97705078125, 
            .............
            0.947265625, 0.9375, 0.90185546875, 0.9501953125, 0.9482421875, 
            ............]), ('Therefore,', 753, 763, 'O', [0.67578125, 0.6533203125]), 
            ("I believe listening to other's advice can help someone make a better choice", 
            764, 839, 'Position', [0.468017578125, 0.75048828125, 0.6181640625, 0.9365234375, 
            ............
            0.93408203125])]
            """
        for phrase_idx, (phrase, start, end, label, phrase_scores) in enumerate(phrase_preds):
            word_start = len(sample_text[:start].split())
            word_end = word_start + len(sample_text[start:end].split())
            word_end = min(word_end, len(sample_text.split()))
            #这个情况非常的严谨
            ps = " ".join([str(x) for x in range(word_start, word_end)])
            r"""
            phrase_idx = 0,phrase = 80% of Americans believe...,and giving
            the hero more choices to consider.
            start = 0,end = 752,label = Lead
            phrase_scores = [0.979,0.980,0.981,...0.943,0.941,0.869]
            word start = 0,word end = 124,ps = 0,1,2,...123
            注意这里总共切分出来的单词为147个(len(phrase_scores) = 147，
            而用空格切分出来的单词为124个，这正是差异的所在。有些单词后面带标点符号，
            这在longformer切分的过程之中会专门使用一格
            这里面之所以使用空格来切分，是因为提交中的predictstring是用空格来切分的
            """
            if label != "O":
                if sum(phrase_scores) / len(phrase_scores) >= proba_thresh[label]:
                    #sum phrase_scores = 140.2255,len phrase_scores = 147,proba_thresh[label] = 0.7
                    #求出每一个phrase的平均得分，判断是否大于proba_thresh[label]的得分，
                    if len(ps.split()) >= min_thresh[label]:
                        #平均得分大于proba_thres[label]并且连续单词数量大于min_thresh[label]时放入最终结果中
                        temp_df.append((sample_id, label, ps))
            
            #if label != "O" and ps != "" and len(phrase_scores) > 2:
            #    temp_df.append((sample_id, label, ps))
    #这里调用对于phrase_preds进行进一步处理
    #提交中需要sample_id,label,ps(用空格切分的内容)
    temp_df = pd.DataFrame(temp_df, columns=["id", "class", "predictionstring"])
    temp_df = temp_df.reset_index(drop=True)
    return temp_df

def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]

def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined.to_csv('joined.csv')
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    joined.to_csv('joined.csv')
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values
    
    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]
    
    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score


target_map_number = {'Lead':0, 'Position':1, 'Evidence':2, 'Claim':3, 'Concluding Statement':4,
             'Counterclaim':5, 'Rebuttal':6}
target_map_number = {'B-Lead':0,'I-Lead':1,'B-Position':2,'I-Position':3,'B-Evidence':4,'I-Evidence':5,\
                     'B-Claim':6,'I-Claim':7,'B-Concluding Statement':8,'I-Concluding Statement':9,\
                     'B-Counterclaim':10,'I-Counterclaim':11,'B-Rebuttal':12,'I-Rebuttal':13,'O':14}
number_map_target = {0:'B-Lead',1:'I-Lead',2:'B-Position',3:'I-Position',4:'B-Evidence',5:'I-Evidence',\
                     6:'B-Claim',7:'I-Claim',8:'B-Concluding Statement',9:'I-Concluding Statement',\
                     10:'B-Counterclaim',11:'I-Counterclaim',12:'B-Rebuttal',13:'I-Rebuttal',14:'O'}


(144293, 8)
There are 15594 train texts.


  0%|                                       | 12/15594 [00:00<02:12, 117.60it/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


  0%|                                       | 36/15594 [00:00<02:11, 118.59it/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


...

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


100%|███████████████████████████████████▉| 15571/15594 [02:17<00:00, 108.15it/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


100%|████████████████████████████████████| 15594/15594 [02:17<00:00, 113.06it/s]

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction


In [2]:
class LabelSmoothingCrossEntropy(nn.Module):
    def __init__(self, eps=0.1, reduction='mean', ignore_index=-100):
        super(LabelSmoothingCrossEntropy, self).__init__()
        self.eps = eps
        self.reduction = reduction
        self.ignore_index = ignore_index

    def forward(self, output, target):
        c = output.size()[-1]
        log_pred = torch.log_softmax(output, dim=-1)
        if self.reduction == 'sum':
            loss = -log_pred.sum()
        else:
            loss = -log_pred.sum(dim=-1)
            if self.reduction == 'mean':
                loss = loss.mean()
        return loss * self.eps / c + (1 - self.eps) * torch.nn.functional.nll_loss(log_pred, target,
                                                                                   reduction=self.reduction,
                                                                                   ignore_index=self.ignore_index)


In [3]:
def compute_multilabel_loss(model,batch_token_ids,batch_attention_mask,batch_label):
    #print('compute_multilabel_loss')
    logit = model(input_ids=batch_token_ids,attention_mask=batch_attention_mask)
    #loss_fn = torch.nn.CrossEntropyLoss(reduce=True, size_average=True)
    #使用CrossEntropyLoss()时网络层不需要加上激活函数softmax
    loss_fn = LabelSmoothingCrossEntropy()
    
    #active_loss = batch_attention_mask.view(-1) == 1
    logit = logit.view(logit.size()[0]*logit.size()[1],-1)
    batch_label = batch_label.view(batch_label.size()[0]*batch_label.size()[1],-1)
    batch_label = batch_label.squeeze()
    
    #outputs = logit.argmax(dim=-1)
    #idxs = np.where(active_loss.cpu().numpy() == 1)[0]
    #logit = logit[idxs]
    #batch_label = batch_label[idxs].to(torch.long)
    r"""
    judge_label1 = (batch_label == 13)
    new_batch_label = torch.zeros_like(batch_label)-100
    new_batch_label = new_batch_label+judge_label1*113
    judge_label2 = (batch_label == 12)
    new_batch_label = new_batch_label+judge_label2*112
    
    mseloss1 = loss_fn(logit,batch_label)
    mseloss2 = loss_fn(logit,new_batch_label)
    mseloss = mseloss1+0.1*mseloss2
    """
    mseloss = loss_fn(logit,batch_label)
    if 12 in batch_label:
        mseloss = mseloss*1.2
    #增加一波标签为12和13的内容
    return mseloss

## 测试学习率

In [4]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from transformers import LongformerModel,LongformerForTokenClassification
from loader_pretrain_longformer import load_longformer_data
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
total_split = 10
total_train_idx = []
total_valid_idx = []

dfx = pd.get_dummies(df, columns=["discourse_type"]).groupby(["id"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("discourse_type_") or c == "id" and c != "discourse_type_num"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "id"]
dfx_labels = dfx[labels]
dfx["kfold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    total_train_idx.append(trn_)
    total_valid_idx.append(val_)

best_point_result = [0.0]*total_split
#for current_split in range(1,total_split):
for current_split in range(0,1):
    train_idx = total_train_idx[current_split]
    valid_idx = total_valid_idx[current_split]
    valid_df = df[df['id'].isin(IDS[valid_idx,])]
    #易漏，每次需要定义valid_df
    
    LongFormer,longformerconfig,get_data = get_model_function('longformer-base')
    config = longformerconfig(**json_data)
    longformer = LongFormer(config)
    #longformer = get_data(longformer,'/home/xiaoguzai/模型/Longformer/pytorch_model.bin')
    longformer = load_longformer_data(longformer,'/home/xiaoguzai/数据/Evaluate student writing/预训练/labeled_data+model_epoch=40.pth')
    model = ClassificationModel(longformer,config,15)
    
    model.train()
    model = model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(),lr=2.5e-5)
    def lr_lambda(epoch):
        if epoch > 5:
            return 1
        else:
            return 2/(epoch+1)
    scheduler = LambdaLR(optimizer, lr_lambda)
    
    train_dataset = TrainDataset(IDS[train_idx,],\
                                train_tokens[train_idx,],\
                                np.array(train_text)[train_idx,],\
                                np.array(train_offset)[train_idx,],\
                                np.array(train_attention)[train_idx,],\
                                targets[train_idx,])
    valid_dataset = TrainDataset(IDS[valid_idx,],\
                                train_tokens[valid_idx,],\
                                np.array(train_text)[valid_idx,],\
                                np.array(train_offset)[valid_idx,],\
                                np.array(train_attention)[valid_idx,],\
                                targets[valid_idx,])
    
    train_loader = DataLoader(train_dataset,batch_size=4,shuffle=True)
    valid_loader = DataLoader(valid_dataset,batch_size=4)
    
    bestpoint = 0.0
    
    for epoch in range(5):
        
        train_loss = 0.0
        print('epoch {}'.format(epoch))
        for batch_ids,batch_token,batch_text,batch_offset,batch_attention_mask,batch_label in tqdm(train_loader):
            batch_token = batch_token.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_label = batch_label.to(device)
            optimizer.zero_grad()
            loss = compute_multilabel_loss(model,batch_token,batch_attention_mask,batch_label)
            #loss, logits = model(input_ids=batch_token, attention_mask=batch_attention_mask, labels=batch_label,
            #                   return_dict=False)
            train_loss = train_loss+loss
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        
        #print("第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
        #注意必须从epoch=1开始，否则第0个没有学习率
        #print('Train Loss: {:.6f}'.format(train_loss))
        #model = torch.load('best_point=0.6460049200792568_current_split=0.pth')
        
        model.eval()
        valid_index = 0
        pred_df = []

        for batch_ids,batch_token,batch_text,batch_offset,batch_attention_mask,batch_label in tqdm(valid_loader):
            batch_token = batch_token.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_label = batch_label.to(device)
            with torch.no_grad():
                output = model(batch_token,attention_mask=batch_attention_mask)
                #output_label = np.argmax(output,axis=-1)
                output = torch.softmax(output,axis=-1)
                output_pred, output_idx = output.max(-1)
                batch_size = output.size()[0]
                r"""
                number_map_target = {0:'B-Lead',1:'I-Lead',2:'B-Position',3:'I-Position',4:'B-Evidence',5:'I-Evidence',\
                     6:'B-Claim',7:'I-Claim',8:'B-Concluding Statement',9:'I-Concluding Statement',\
                     10:'B-Counterclaim',11:'I-Counterclaim',12:'B-Rebuttal',13:'I-Rebuttal',14:'O'}
                """
                for index1 in range(batch_size):
                    current_pred = [number_map_target[int(data)] for data in output_idx[index1]]
                    r"""
                    pred_df.append({'id':batch_ids[index1],\
                                   'input_ids':batch_token[index1],\
                                   'text':batch_text[index1],\
                                   'offset_mapping':batch_offset[index1],\
                                   'preds':current_pred,
                                   'pred_scores':output_idx[index1]})
                    """
                    pred_df.append({'id':batch_ids[index1],\
                                    'input_ids':batch_token[index1][1:],\
                                    'text':batch_text[index1],\
                                    'offset_mapping':batch_offset[index1][1:],\
                                    'preds':current_pred[1:],\
                                    'pred_scores':output_pred[index1][1:]})
                    #预测出来的结果去掉开头的[cls]标志,容易出现bug的地方：text不需要去除第一位的字母
                    #其他的像offset_mapping,preds,pred_scores等可以
                    #batch_ids不能带[1:]，为文章的id
        prepred_df = pred_df
        pred_df = change_id_to_predictstring(pred_df)

        CLASSES = pred_df['class'].unique()
        f1s = []

        for c in CLASSES:
            current_pred_df = pred_df.loc[pred_df['class']==c].copy()
            current_valid_df = valid_df.loc[valid_df['discourse_type']==c].copy()
            f1 = score_feedback_comp(current_pred_df,current_valid_df)
        #point = score_feedback_comp(pred_df,valid_df)
            print(c,f1)
            f1s.append(f1)
        point = np.mean(f1s)

        #point = score_feedback_comp(pred_df,valid_df)
        print('point = %s'%(str(point)))
        if bestpoint < point:
            bestpoint = point
            print('bestpoint = %s'%(str(bestpoint)))
            torch.save(model,'best_point='+str(bestpoint)+'_current_split='+str(current_split)+'.pth')
            best_point_result[current_split] = bestpoint
    break

Done loading 269 longformer weights from: /home/xiaoguzai/数据/Evaluate student writing/预训练/labeled_data+model_epoch=40.pth. Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	module.prediction_dense.weight
	module.prediction_layer_norm.weight
	module.prediction_dense.bias
	module.decoder_bias
	module.prediction_layer_norm.bias
	module.prediction_decoder.bias
	module.prediction_decoder.weight
epoch 0


100%|█████████████████████████████████████████| 390/390 [01:36<00:00,  4.05it/s]


Lead 0.6519706078824316
Position 0.6673511293634496
Claim 0.6039432722241439
Evidence 0.7080442984057442
Concluding Statement 0.8031683168316832
Counterclaim 0.49518810148731407
Rebuttal 0.3659574468085106
point = 0.6136604532861825
bestpoint = 0.6136604532861825
epoch 1


100%|█████████████████████████████████████████| 390/390 [01:36<00:00,  4.04it/s]


Lead 0.7603911980440098
Position 0.6891701828410689
Claim 0.6005284650492433
Evidence 0.7349411904479031
Concluding Statement 0.8138319260152794
Counterclaim 0.4611872146118721
Rebuttal 0.35702479338842974
point = 0.6310107100568295
bestpoint = 0.6310107100568295
epoch 2


...

Lead 0.806978052898143
Position 0.6864922966678609
Claim 0.6036542515811666
Evidence 0.7338862559241706
Concluding Statement 0.8389081122645137
Counterclaim 0.4881422924901186
Rebuttal 0.43441466854724964
point = 0.6560679900533176
bestpoint = 0.6560679900533176
epoch 3


100%|█████████████████████████████████████████| 390/390 [01:37<00:00,  4.00it/s]


Lead 0.8105726872246696
Position 0.6945404284727021
Claim 0.6123778501628665
Evidence 0.7333257584365414
Concluding Statement 0.8364732479276563
Counterclaim 0.49085659287776706
Rebuttal 0.4160887656033287
point = 0.6563193329579331
bestpoint = 0.6563193329579331
epoch 4


100%|█████████████████████████████████████████| 390/390 [01:36<00:00,  4.06it/s]


Lead 0.8114035087719298
Position 0.6769540422676954
Claim 0.6176813128639492
Evidence 0.7317019400352733
Concluding Statement 0.8276899924755455
Counterclaim 0.49544626593806923
Rebuttal 0.39588688946015427
point = 0.650966278830374


In [5]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from transformers import LongformerModel,LongformerForTokenClassification
from loader_pretrain_longformer import load_longformer_data
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
total_split = 10
total_train_idx = []
total_valid_idx = []

dfx = pd.get_dummies(df, columns=["discourse_type"]).groupby(["id"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("discourse_type_") or c == "id" and c != "discourse_type_num"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "id"]
dfx_labels = dfx[labels]
dfx["kfold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    total_train_idx.append(trn_)
    total_valid_idx.append(val_)

best_point_result = [0.0]*total_split
#for current_split in range(1,total_split):
for current_split in range(0,1):
    train_idx = total_train_idx[current_split]
    valid_idx = total_valid_idx[current_split]
    valid_df = df[df['id'].isin(IDS[valid_idx,])]
    #易漏，每次需要定义valid_df
    
    LongFormer,longformerconfig,get_data = get_model_function('longformer-base')
    config = longformerconfig(**json_data)
    longformer = LongFormer(config)
    #longformer = get_data(longformer,'/home/xiaoguzai/模型/Longformer/pytorch_model.bin')
    longformer = load_longformer_data(longformer,'/home/xiaoguzai/数据/Evaluate student writing/预训练/labeled_data+model_epoch=40.pth')
    model = ClassificationModel(longformer,config,15)
    
    model.train()
    model = model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(),lr=1.5e-5)
    def lr_lambda(epoch):
        if epoch > 5:
            return 1
        else:
            return 2/(epoch+1)
    scheduler = LambdaLR(optimizer, lr_lambda)
    
    train_dataset = TrainDataset(IDS[train_idx,],\
                                train_tokens[train_idx,],\
                                np.array(train_text)[train_idx,],\
                                np.array(train_offset)[train_idx,],\
                                np.array(train_attention)[train_idx,],\
                                targets[train_idx,])
    valid_dataset = TrainDataset(IDS[valid_idx,],\
                                train_tokens[valid_idx,],\
                                np.array(train_text)[valid_idx,],\
                                np.array(train_offset)[valid_idx,],\
                                np.array(train_attention)[valid_idx,],\
                                targets[valid_idx,])
    
    train_loader = DataLoader(train_dataset,batch_size=4,shuffle=True)
    valid_loader = DataLoader(valid_dataset,batch_size=4)
    
    bestpoint = 0.0
    
    for epoch in range(5):
        
        train_loss = 0.0
        print('epoch {}'.format(epoch))
        for batch_ids,batch_token,batch_text,batch_offset,batch_attention_mask,batch_label in tqdm(train_loader):
            batch_token = batch_token.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_label = batch_label.to(device)
            optimizer.zero_grad()
            loss = compute_multilabel_loss(model,batch_token,batch_attention_mask,batch_label)
            #loss, logits = model(input_ids=batch_token, attention_mask=batch_attention_mask, labels=batch_label,
            #                   return_dict=False)
            train_loss = train_loss+loss
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        
        #print("第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
        #注意必须从epoch=1开始，否则第0个没有学习率
        #print('Train Loss: {:.6f}'.format(train_loss))
        #model = torch.load('best_point=0.6460049200792568_current_split=0.pth')
        
        model.eval()
        valid_index = 0
        pred_df = []

        for batch_ids,batch_token,batch_text,batch_offset,batch_attention_mask,batch_label in tqdm(valid_loader):
            batch_token = batch_token.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_label = batch_label.to(device)
            with torch.no_grad():
                output = model(batch_token,attention_mask=batch_attention_mask)
                #output_label = np.argmax(output,axis=-1)
                output = torch.softmax(output,axis=-1)
                output_pred, output_idx = output.max(-1)
                batch_size = output.size()[0]
                r"""
                number_map_target = {0:'B-Lead',1:'I-Lead',2:'B-Position',3:'I-Position',4:'B-Evidence',5:'I-Evidence',\
                     6:'B-Claim',7:'I-Claim',8:'B-Concluding Statement',9:'I-Concluding Statement',\
                     10:'B-Counterclaim',11:'I-Counterclaim',12:'B-Rebuttal',13:'I-Rebuttal',14:'O'}
                """
                for index1 in range(batch_size):
                    current_pred = [number_map_target[int(data)] for data in output_idx[index1]]
                    r"""
                    pred_df.append({'id':batch_ids[index1],\
                                   'input_ids':batch_token[index1],\
                                   'text':batch_text[index1],\
                                   'offset_mapping':batch_offset[index1],\
                                   'preds':current_pred,
                                   'pred_scores':output_idx[index1]})
                    """
                    pred_df.append({'id':batch_ids[index1],\
                                    'input_ids':batch_token[index1][1:],\
                                    'text':batch_text[index1],\
                                    'offset_mapping':batch_offset[index1][1:],\
                                    'preds':current_pred[1:],\
                                    'pred_scores':output_pred[index1][1:]})
                    #预测出来的结果去掉开头的[cls]标志,容易出现bug的地方：text不需要去除第一位的字母
                    #其他的像offset_mapping,preds,pred_scores等可以
                    #batch_ids不能带[1:]，为文章的id
        prepred_df = pred_df
        pred_df = change_id_to_predictstring(pred_df)

        CLASSES = pred_df['class'].unique()
        f1s = []

        for c in CLASSES:
            current_pred_df = pred_df.loc[pred_df['class']==c].copy()
            current_valid_df = valid_df.loc[valid_df['discourse_type']==c].copy()
            f1 = score_feedback_comp(current_pred_df,current_valid_df)
        #point = score_feedback_comp(pred_df,valid_df)
            print(c,f1)
            f1s.append(f1)
        point = np.mean(f1s)

        #point = score_feedback_comp(pred_df,valid_df)
        print('point = %s'%(str(point)))
        if bestpoint < point:
            bestpoint = point
            print('bestpoint = %s'%(str(bestpoint)))
            torch.save(model,'best_point='+str(bestpoint)+'_current_split='+str(current_split)+'.pth')
            best_point_result[current_split] = bestpoint
    break

Done loading 269 longformer weights from: /home/xiaoguzai/数据/Evaluate student writing/预训练/labeled_data+model_epoch=40.pth. Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	module.prediction_dense.weight
	module.prediction_layer_norm.weight
	module.prediction_dense.bias
	module.decoder_bias
	module.prediction_layer_norm.bias
	module.prediction_decoder.bias
	module.prediction_decoder.weight
epoch 0


100%|█████████████████████████████████████████| 390/390 [01:37<00:00,  4.00it/s]


Lead 0.7643236857649144
Position 0.6754824874910651
Evidence 0.7343714220288527
Claim 0.5468223086900129
Concluding Statement 0.8066383635661907
Counterclaim 0.47152847152847155
Rebuttal 0.294017094017094
point = 0.6133119761552288
bestpoint = 0.6133119761552288
epoch 1


100%|█████████████████████████████████████████| 390/390 [01:36<00:00,  4.06it/s]


Lead 0.73992673992674
Position 0.6946022727272727
Claim 0.5975356837867513
Evidence 0.725250549987778
Concluding Statement 0.8201941747572815
Counterclaim 0.47045707915273133
Rebuttal 0.32550335570469796
point = 0.6247814080061789
bestpoint = 0.6247814080061789
epoch 2


...

Lead 0.7865680136596471
Position 0.6898002103049422
Claim 0.6097817205556204
Evidence 0.7373880245167374
Concluding Statement 0.8474204171240395
Counterclaim 0.47058823529411764
Rebuttal 0.38580246913580246
point = 0.646764155798701
bestpoint = 0.646764155798701
epoch 3


100%|█████████████████████████████████████████| 390/390 [01:36<00:00,  4.06it/s]


Lead 0.7993402968664101
Position 0.691156462585034
Claim 0.6235357334766255
Evidence 0.7295081967213115
Concluding Statement 0.8264150943396227
Counterclaim 0.5148895292987512
Rebuttal 0.3767313019390582
point = 0.651653802175259
bestpoint = 0.651653802175259
epoch 4


100%|█████████████████████████████████████████| 390/390 [01:36<00:00,  4.05it/s]


Lead 0.7935447968836951
Position 0.6869300911854104
Claim 0.6096055247462593
Evidence 0.7235043692583464
Concluding Statement 0.8355754857997011
Counterclaim 0.4831013916500994
Rebuttal 0.36441893830703015
point = 0.6423829425472202


In [7]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from transformers import LongformerModel,LongformerForTokenClassification
from loader_pretrain_longformer import load_longformer_data
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
total_split = 10
total_train_idx = []
total_valid_idx = []

dfx = pd.get_dummies(df, columns=["discourse_type"]).groupby(["id"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("discourse_type_") or c == "id" and c != "discourse_type_num"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "id"]
dfx_labels = dfx[labels]
dfx["kfold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    total_train_idx.append(trn_)
    total_valid_idx.append(val_)

best_point_result = [0.0]*total_split
#for current_split in range(1,total_split):
for current_split in range(0,1):
    train_idx = total_train_idx[current_split]
    valid_idx = total_valid_idx[current_split]
    valid_df = df[df['id'].isin(IDS[valid_idx,])]
    #易漏，每次需要定义valid_df
    
    LongFormer,longformerconfig,get_data = get_model_function('longformer-base')
    config = longformerconfig(**json_data)
    longformer = LongFormer(config)
    #longformer = get_data(longformer,'/home/xiaoguzai/模型/Longformer/pytorch_model.bin')
    longformer = load_longformer_data(longformer,'/home/xiaoguzai/数据/Evaluate student writing/预训练/labeled_data+model_epoch=40.pth')
    model = ClassificationModel(longformer,config,15)
    
    model.train()
    model = model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(),lr=3e-5)
    def lr_lambda(epoch):
        if epoch > 5:
            return 1
        else:
            return 2/(epoch+1)
    scheduler = LambdaLR(optimizer, lr_lambda)
    
    train_dataset = TrainDataset(IDS[train_idx,],\
                                train_tokens[train_idx,],\
                                np.array(train_text)[train_idx,],\
                                np.array(train_offset)[train_idx,],\
                                np.array(train_attention)[train_idx,],\
                                targets[train_idx,])
    valid_dataset = TrainDataset(IDS[valid_idx,],\
                                train_tokens[valid_idx,],\
                                np.array(train_text)[valid_idx,],\
                                np.array(train_offset)[valid_idx,],\
                                np.array(train_attention)[valid_idx,],\
                                targets[valid_idx,])
    
    train_loader = DataLoader(train_dataset,batch_size=4,shuffle=True)
    valid_loader = DataLoader(valid_dataset,batch_size=4)
    
    bestpoint = 0.0
    
    for epoch in range(5):
        
        train_loss = 0.0
        print('epoch {}'.format(epoch))
        for batch_ids,batch_token,batch_text,batch_offset,batch_attention_mask,batch_label in tqdm(train_loader):
            batch_token = batch_token.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_label = batch_label.to(device)
            optimizer.zero_grad()
            loss = compute_multilabel_loss(model,batch_token,batch_attention_mask,batch_label)
            #loss, logits = model(input_ids=batch_token, attention_mask=batch_attention_mask, labels=batch_label,
            #                   return_dict=False)
            train_loss = train_loss+loss
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        
        #print("第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
        #注意必须从epoch=1开始，否则第0个没有学习率
        #print('Train Loss: {:.6f}'.format(train_loss))
        #model = torch.load('best_point=0.6460049200792568_current_split=0.pth')
        
        model.eval()
        valid_index = 0
        pred_df = []

        for batch_ids,batch_token,batch_text,batch_offset,batch_attention_mask,batch_label in tqdm(valid_loader):
            batch_token = batch_token.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_label = batch_label.to(device)
            with torch.no_grad():
                output = model(batch_token,attention_mask=batch_attention_mask)
                #output_label = np.argmax(output,axis=-1)
                output = torch.softmax(output,axis=-1)
                output_pred, output_idx = output.max(-1)
                batch_size = output.size()[0]
                r"""
                number_map_target = {0:'B-Lead',1:'I-Lead',2:'B-Position',3:'I-Position',4:'B-Evidence',5:'I-Evidence',\
                     6:'B-Claim',7:'I-Claim',8:'B-Concluding Statement',9:'I-Concluding Statement',\
                     10:'B-Counterclaim',11:'I-Counterclaim',12:'B-Rebuttal',13:'I-Rebuttal',14:'O'}
                """
                for index1 in range(batch_size):
                    current_pred = [number_map_target[int(data)] for data in output_idx[index1]]
                    r"""
                    pred_df.append({'id':batch_ids[index1],\
                                   'input_ids':batch_token[index1],\
                                   'text':batch_text[index1],\
                                   'offset_mapping':batch_offset[index1],\
                                   'preds':current_pred,
                                   'pred_scores':output_idx[index1]})
                    """
                    pred_df.append({'id':batch_ids[index1],\
                                    'input_ids':batch_token[index1][1:],\
                                    'text':batch_text[index1],\
                                    'offset_mapping':batch_offset[index1][1:],\
                                    'preds':current_pred[1:],\
                                    'pred_scores':output_pred[index1][1:]})
                    #预测出来的结果去掉开头的[cls]标志,容易出现bug的地方：text不需要去除第一位的字母
                    #其他的像offset_mapping,preds,pred_scores等可以
                    #batch_ids不能带[1:]，为文章的id
        prepred_df = pred_df
        pred_df = change_id_to_predictstring(pred_df)

        CLASSES = pred_df['class'].unique()
        f1s = []

        for c in CLASSES:
            current_pred_df = pred_df.loc[pred_df['class']==c].copy()
            current_valid_df = valid_df.loc[valid_df['discourse_type']==c].copy()
            f1 = score_feedback_comp(current_pred_df,current_valid_df)
        #point = score_feedback_comp(pred_df,valid_df)
            print(c,f1)
            f1s.append(f1)
        point = np.mean(f1s)

        #point = score_feedback_comp(pred_df,valid_df)
        print('point = %s'%(str(point)))
        if bestpoint < point:
            bestpoint = point
            print('bestpoint = %s'%(str(bestpoint)))
            torch.save(model,'best_point='+str(bestpoint)+'_current_split='+str(current_split)+'.pth')
            best_point_result[current_split] = bestpoint
    break

Done loading 269 longformer weights from: /home/xiaoguzai/数据/Evaluate student writing/预训练/labeled_data+model_epoch=40.pth. Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	module.prediction_dense.weight
	module.prediction_layer_norm.weight
	module.prediction_dense.bias
	module.decoder_bias
	module.prediction_layer_norm.bias
	module.prediction_decoder.bias
	module.prediction_decoder.weight
epoch 0


100%|█████████████████████████████████████████| 390/390 [01:27<00:00,  4.45it/s]


Lead 0.7915476870359794
Position 0.6395604395604395
Evidence 0.7072691552062869
Concluding Statement 0.7847896440129449
Claim 0.461326727222681
Counterclaim 0.35599505562422745
Rebuttal 0.15578947368421053
point = 0.5566111689066814
bestpoint = 0.5566111689066814
epoch 1


 49%|███████████████████▏                   | 1721/3509 [16:49<17:28,  1.70it/s]


KeyboardInterrupt: 

In [ ]:
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR
from transformers import LongformerModel,LongformerForTokenClassification
from loader_pretrain_longformer import load_longformer_data
from sklearn.model_selection import KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
total_split = 10
total_train_idx = []
total_valid_idx = []

dfx = pd.get_dummies(df, columns=["discourse_type"]).groupby(["id"], as_index=False).sum()
cols = [c for c in dfx.columns if c.startswith("discourse_type_") or c == "id" and c != "discourse_type_num"]
dfx = dfx[cols]

mskf = MultilabelStratifiedKFold(n_splits=10, shuffle=True, random_state=42)
labels = [c for c in dfx.columns if c != "id"]
dfx_labels = dfx[labels]
dfx["kfold"] = -1

for fold, (trn_, val_) in enumerate(mskf.split(dfx, dfx_labels)):
    total_train_idx.append(trn_)
    total_valid_idx.append(val_)

best_point_result = [0.0]*total_split
#for current_split in range(1,total_split):
for current_split in range(0,1):
    train_idx = total_train_idx[current_split]
    valid_idx = total_valid_idx[current_split]
    valid_df = df[df['id'].isin(IDS[valid_idx,])]
    #易漏，每次需要定义valid_df
    
    LongFormer,longformerconfig,get_data = get_model_function('longformer-base')
    config = longformerconfig(**json_data)
    longformer = LongFormer(config)
    #longformer = get_data(longformer,'/home/xiaoguzai/模型/Longformer/pytorch_model.bin')
    longformer = load_longformer_data(longformer,'/home/xiaoguzai/数据/Evaluate student writing/预训练/labeled_data+model_epoch=40.pth')
    model = ClassificationModel(longformer,config,15)
    
    model.train()
    model = model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(),lr=9e-6)
    def lr_lambda(epoch):
        if epoch > 5:
            return 1
        else:
            return 2/(epoch+1)
    scheduler = LambdaLR(optimizer, lr_lambda)
    
    train_dataset = TrainDataset(IDS[train_idx,],\
                                train_tokens[train_idx,],\
                                np.array(train_text)[train_idx,],\
                                np.array(train_offset)[train_idx,],\
                                np.array(train_attention)[train_idx,],\
                                targets[train_idx,])
    valid_dataset = TrainDataset(IDS[valid_idx,],\
                                train_tokens[valid_idx,],\
                                np.array(train_text)[valid_idx,],\
                                np.array(train_offset)[valid_idx,],\
                                np.array(train_attention)[valid_idx,],\
                                targets[valid_idx,])
    
    train_loader = DataLoader(train_dataset,batch_size=4,shuffle=True)
    valid_loader = DataLoader(valid_dataset,batch_size=4)
    
    bestpoint = 0.0
    
    for epoch in range(5):
        
        train_loss = 0.0
        print('epoch {}'.format(epoch))
        for batch_ids,batch_token,batch_text,batch_offset,batch_attention_mask,batch_label in tqdm(train_loader):
            batch_token = batch_token.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_label = batch_label.to(device)
            optimizer.zero_grad()
            loss = compute_multilabel_loss(model,batch_token,batch_attention_mask,batch_label)
            #loss, logits = model(input_ids=batch_token, attention_mask=batch_attention_mask, labels=batch_label,
            #                   return_dict=False)
            train_loss = train_loss+loss
            loss.backward()
            optimizer.step()
        scheduler.step()
        
        
        #print("第%d个epoch的学习率：%f" % (epoch, optimizer.param_groups[0]['lr']))
        #注意必须从epoch=1开始，否则第0个没有学习率
        #print('Train Loss: {:.6f}'.format(train_loss))
        #model = torch.load('best_point=0.6460049200792568_current_split=0.pth')
        
        model.eval()
        valid_index = 0
        pred_df = []

        for batch_ids,batch_token,batch_text,batch_offset,batch_attention_mask,batch_label in tqdm(valid_loader):
            batch_token = batch_token.to(device)
            batch_attention_mask = batch_attention_mask.to(device)
            batch_label = batch_label.to(device)
            with torch.no_grad():
                output = model(batch_token,attention_mask=batch_attention_mask)
                #output_label = np.argmax(output,axis=-1)
                output = torch.softmax(output,axis=-1)
                output_pred, output_idx = output.max(-1)
                batch_size = output.size()[0]
                r"""
                number_map_target = {0:'B-Lead',1:'I-Lead',2:'B-Position',3:'I-Position',4:'B-Evidence',5:'I-Evidence',\
                     6:'B-Claim',7:'I-Claim',8:'B-Concluding Statement',9:'I-Concluding Statement',\
                     10:'B-Counterclaim',11:'I-Counterclaim',12:'B-Rebuttal',13:'I-Rebuttal',14:'O'}
                """
                for index1 in range(batch_size):
                    current_pred = [number_map_target[int(data)] for data in output_idx[index1]]
                    r"""
                    pred_df.append({'id':batch_ids[index1],\
                                   'input_ids':batch_token[index1],\
                                   'text':batch_text[index1],\
                                   'offset_mapping':batch_offset[index1],\
                                   'preds':current_pred,
                                   'pred_scores':output_idx[index1]})
                    """
                    pred_df.append({'id':batch_ids[index1],\
                                    'input_ids':batch_token[index1][1:],\
                                    'text':batch_text[index1],\
                                    'offset_mapping':batch_offset[index1][1:],\
                                    'preds':current_pred[1:],\
                                    'pred_scores':output_pred[index1][1:]})
                    #预测出来的结果去掉开头的[cls]标志,容易出现bug的地方：text不需要去除第一位的字母
                    #其他的像offset_mapping,preds,pred_scores等可以
                    #batch_ids不能带[1:]，为文章的id
        prepred_df = pred_df
        pred_df = change_id_to_predictstring(pred_df)

        CLASSES = pred_df['class'].unique()
        f1s = []

        for c in CLASSES:
            current_pred_df = pred_df.loc[pred_df['class']==c].copy()
            current_valid_df = valid_df.loc[valid_df['discourse_type']==c].copy()
            f1 = score_feedback_comp(current_pred_df,current_valid_df)
        #point = score_feedback_comp(pred_df,valid_df)
            print(c,f1)
            f1s.append(f1)
        point = np.mean(f1s)

        #point = score_feedback_comp(pred_df,valid_df)
        print('point = %s'%(str(point)))
        if bestpoint < point:
            bestpoint = point
            print('bestpoint = %s'%(str(bestpoint)))
            torch.save(model,'best_point='+str(bestpoint)+'_current_split='+str(current_split)+'.pth')
            best_point_result[current_split] = bestpoint
    break